In [ ]:
import time


In [ ]:
for i in range(0,5):
    print(i+1, ": Praying to code almighty for a good grade." + "I hope I don't get a meh score.")
    time.sleep(0.6)
print("\n \nHope github doesn't crash.")
print("Hope Huggingface doesn't crash.")
print("Hope my computer doesn't crash.")
print("Hope I don't crash.")
print("Hope all the above prayers work.")

In [ ]:
#install pytorch if you dont have it(cpu/cuda)
#%pip install torch torchvision
#%pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
%pip install transformers
%pip install scipy
%pip install gradio


In [ ]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
    
demo.launch()  

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

The model was trained when username was denoted as @user and the links are replaced with http. So we bow down to the arbitary model throwing tantrums and satisfy its wishes

In [ ]:

# preprocess tweet as function
def preprocess(tweet):
    tweet_words = []

    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)
    return tweet_proc


In [ ]:
# load model and tokenizer
#roberta = "cardiffnlp/twitter-roberta-base-sentiment" this is the base model
roberta = "sudhanvasp/Sentiment-Analysis" #this is the custom model that utilizes the base model with additional parameters to improve accuracy and performance


model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']


In [ ]:
#output as a function
def sentiment(encoded_tweet):
        tweet_proc = preprocess(tweet)
        encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

        output = model(**encoded_tweet)

        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        #storing labels and scores in a dictionary in one line
        results = dict(zip(labels, scores))
         #print max score and label of the tweet
        print("Tweet: ", tweet)
        print("Sentiment: ", labels[scores.argmax()], scores.max())
        print("\n\n")
        print("In detail: ")
        return results

       

        
    

In [ ]:
for i in range(0,5):
    tweet = input("Enter a tweet: ")
    tweet_proc = preprocess(tweet)
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
    sentiment(encoded_tweet)
   